Import libraries

In [8]:
import numpy as np
import pandas as pd
from pyspark.sql import Window
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import *

Create SparkSession

In [9]:
spark = SparkSession \
    .builder \
    .appName("Steam") \
    .getOrCreate()

Schema Dataframe

In [10]:
schema = StructType([
    StructField("appid", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("release_date", StringType(), True),
    StructField("developer", IntegerType(), True),
    StructField("publisher", StringType(), True),
    StructField("platforms", IntegerType(), True),
    StructField("categories", StringType(), True),
    StructField("genres", IntegerType(), True),
    StructField("positive_ratings", IntegerType(), True),
    StructField("negative_ratings", IntegerType(), True),
    StructField("average_playtime", IntegerType(), True),
    StructField("median_playtime", IntegerType(), True),
    StructField("owners", StringType(), True),
    StructField("min_owners", IntegerType(), True),
    StructField("max_owners", IntegerType(), True),
    StructField("price", FloatType(), True)
])

Download DataFrame from Hadoop data source

In [11]:
df = spark.read.csv('hdfs://localhost:9000/esgi/steam/steam2.csv',
                    header='true',
                    schema=schema,
                    sep=";")

In [12]:
df = df.na.fill(0)

Les studios de dév avec le plus de joueurs

In [13]:
dfg = df.groupBy('developer').agg({'max_owners' : 'sum'})
dfPandas = dfg.sort(desc('sum(max_owners)'))
dfPandas.toPandas().head(10)

,developer,sum(max_owners)
0,0,1913160000
1,773,100000


Les studios avec le moins de joueurs

In [14]:
dfg = df.groupBy('developer').agg({'max_owners' : 'sum'})
dfPandas = dfg.sort(asc('sum(max_owners)'))
dfPandas.toPandas().head(10)

,developer,sum(max_owners)
0,773,100000
1,0,1913160000


Nombre de jeux par catégories

In [15]:
dfg = df.groupBy('genres').agg({'name' : 'count'})
dfgPandas = dfg.sort(desc('count(name)'))
dfgPandas.toPandas().head(10)

,genres,count(name)
0,0,1509


Prix moyen d'un jeu

In [16]:
df.agg({'price' : 'mean'}).toPandas()

,avg(price)
0,7.656401


Les 10 jeux avec plus d'avis positif par catégories

In [17]:
dtg = df.groupBy('genres', 'name').agg({'positive_ratings' : 'max'})
dtgPandasDesc = dtg.sort(desc('max(positive_ratings)'))
dtgPandasDesc.toPandas().head(10)

,genres,name,max(positive_ratings)
0,0,Counter-Strike: Global Offensive,2644404
1,0,Dota 2,863507
2,0,Team Fortress 2,515879
3,0,Garry's Mod,363721
4,0,PAYDAY 2,308657
5,0,Terraria,255600
6,0,Left 4 Dead 2,251789
7,0,The Elder Scrolls V: Skyrim,237303
8,0,Borderlands 2,144595
9,0,Portal 2,138220


Les 10 jeux avec plus d'avis négatif par catégories

In [18]:
dtg = df.groupBy('genres', 'name').agg({'negative_ratings' : 'max'})
dtgPandasDesc = dtg.sort(desc('max(negative_ratings)'))
dtgPandasDesc.toPandas().head(10)

,genres,name,max(negative_ratings)
0,0,Counter-Strike: Global Offensive,402313
1,0,Dota 2,142079
2,0,DayZ,77169
3,0,PAYDAY 2,56523
4,0,Team Fortress 2,34036
5,0,Grand Theft Auto IV,18926
6,0,Garry's Mod,16433
7,0,The Elder Scrolls V: Skyrim,14951
8,0,Total War™: ROME II - Emperor Edition,13423
9,0,Batman™: Arkham Knight,12981


Moyenne des temps de jeux par genre

In [19]:
dfg = df.groupBy('genres').agg({'median_playtime' : 'avg'})
dfgPanda = dfg.sort(desc('avg(median_playtime)'))
dfgPanda.toPandas().head(10)

,genres,avg(median_playtime)
0,0,390.526839


Les 10 jeux les plus couteux

In [20]:
dfg = df.groupBy('name').agg({'price' : 'max'})
dfgPandas = dfg.sort(desc('max(price)'))
dfgPandas.toPandas().head(10)

,name,max(price)
0,3DCoat 4.8,95.989998
1,RPG Maker VX Ace,52.990002
2,Call of Duty®: Black Ops II,39.990002
3,Call of Duty®: Ghosts,39.990002
4,Call of Duty®: Advanced Warfare - Gold Edition,39.990002
5,F.E.A.R.,35.959999
6,Train Simulator 2019,34.990002
7,Ironclads: High Seas,34.950001
8,Ironclads: Chincha Islands War 1866,34.950001
9,Ironclads: American Civil War,34.950001


Meilleur ratio des jeux en fonction des avis positifs et négatifs

In [21]:
rdf = df.withColumn('ratio', col("positive_ratings") / (col("positive_ratings") + col("negative_ratings")))
rdf.select(['name', 'ratio']).toPandas().head(10)

,name,ratio
0,Counter-Strike,0.973888
1,Team Fortress Classic,0.839787
2,Day of Defeat,0.895648
3,Deathmatch Classic,0.826623
4,Half-Life: Opposing Force,0.947996
5,Ricochet,0.801278
6,Half-Life,0.961878
7,Counter-Strike: Condition Zero,0.893871
8,Half-Life: Blue Shift,0.900990
9,Half-Life 2,0.965601


Médiane des prix par genres

In [22]:
dfg = df.groupBy('genres').agg(percentile_approx(col('price'), 0.5))
dfg.toPandas()

,genres,"percentile_approx(price, 0.5, 10000)"
0,0,6.99


Partie Clustering (k-means)

In [23]:
df.groupBy('genres').count().toPandas()

,genres,count
0,0,1509


In [24]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import matplotlib.pyplot as plt

In [25]:
imput_cols = [
 'platforms',
 'developer',
 'genres',
 'median_playtime',
 'min_owners',
 'max_owners',
 'price']

vector_assembler = VectorAssembler(inputCols=imput_cols, outputCol="features")

final_df = vector_assembler.transform(df)

KMeans

In [26]:
kmeans = KMeans(featuresCol='features', k=30)
model = kmeans.fit(final_df)

In [27]:
model.transform(final_df).groupBy('prediction').count().toPandas()

,prediction,count
0,28,3
1,27,17
2,26,5
3,12,4
4,22,38
5,1,1
6,13,4
7,6,37
8,16,2
9,3,1


Prediction c'est le cluster count c'est le nombre de jeux

In [28]:
predictions = model.transform(final_df)
predictions.filter(predictions.prediction == "26").toPandas().head(20)

,appid,name,release_date,developer,publisher,platforms,categories,genres,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,min_owners,max_owners,price,features,prediction
0,12210,Grand Theft Auto IV,02/12/2008,0,Rockstar Games,0,Single-player;Multi-player;Partial Controller ...,0,35240,18926,1255,634,2000000-5000000,2000000,5000000,19.99,"[0.0, 0.0, 0.0, 634.0, 2000000.0, 5000000.0, 1...",26
1,24960,Battlefield: Bad Company™ 2,02/03/2010,0,Electronic Arts,0,Single-player;Multi-player,0,23093,3877,1200,671,2000000-5000000,2000000,5000000,14.99,"[0.0, 0.0, 0.0, 671.0, 2000000.0, 5000000.0, 1...",26
2,200260,Batman: Arkham City - Game of the Year Edition,07/09/2012,0,Warner Bros. Interactive Entertainment;Feral I...,0,Single-player;Steam Achievements;Steam Trading...,0,25995,1255,1554,618,2000000-5000000,2000000,5000000,14.99,"[0.0, 0.0, 0.0, 618.0, 2000000.0, 5000000.0, 1...",26
3,212680,FTL: Faster Than Light,14/09/2012,0,Subset Games,0,Single-player;Steam Trading Cards,0,42641,1629,1111,633,2000000-5000000,2000000,5000000,6.99,"[0.0, 0.0, 0.0, 633.0, 2000000.0, 5000000.0, 6...",26
4,213670,South Park™: The Stick of Truth™,06/03/2014,0,Ubisoft,0,Single-player;Steam Achievements;Full controll...,0,35896,911,1045,628,2000000-5000000,2000000,5000000,25.99,"[0.0, 0.0, 0.0, 628.0, 2000000.0, 5000000.0, 2...",26
